In [0]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torchsummary import summary

In [55]:
print("--sys.version--")
print(sys.version)

print("\n--torch.__version__--")
print(torch.__version__)

--sys.version--
3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]

--torch.__version__--
1.1.0


In [0]:
batch_size = 20
total_epoch = 1000
learning_rate = 0.1
use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()

In [57]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/train", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/test", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
def train(model, train_loader):
  model.train()
  
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
  losses = []
  for i, (image, label) in enumerate(train_loader):
    
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
    pred_label = model(image)
    loss = criterion(pred_label, label)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  avg_loss = sum(losses)/len(losses)
  return avg_loss

In [0]:
def eval(model, test_loader):
  model.eval()
  device = next(model.parameters()).device.index
  
  total_cnt = 0
  correct_cnt = 0
  
  for i, (image, label) in enumerate(test_loader):
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
      out = model(image)
      _, pred_label = torch.max(out.data, 1)
      total_cnt += image.data.size()[0]
      correct_cnt += (pred_label == label.data).sum().item()
      
    return correct_cnt / total_cnt

In [0]:
class SimpleMLP(nn.Module):
  def __init__(self):
    super(SimpleMLP, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.ReLU()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.ReLU()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.ReLU()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.ReLU()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
class SimpleMLP_Sigmoid(nn.Module):
  def __init__(self):
    super(SimpleMLP_Sigmoid, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.Sigmoid()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.Sigmoid()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.Sigmoid()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.Sigmoid()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    # Convolution layer
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2)
    
    self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2)
    
    self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
    self.act3 = nn.ReLU()
    
    self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
    self.act4 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2)
    
    # Fully-Connected layer
    self.fc1 = nn.Linear(256 * 2 * 2, 1000)
    self.act5 = nn.ReLU()
    self.output = nn.Linear(1000, 10)
    
    
  def forward(self, x):
    x = self.pool1(self.act1(self.conv1(x)))
    x = self.pool2(self.act2(self.conv2(x)))
    x = self.act3(self.conv3(x))
    x = self.act4(self.conv4(x))
    x = self.pool3(x)
    
    x = x.view(-1, 256 * 2 * 2)
    
    x = self.act5(self.fc1(x))
    out = self.output(x)
    return out

In [0]:
class SimpleVGG(nn.Module):
  def __init__(self):
    super(SimpleVGG, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), padding=(1,1))
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    
    self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=(1,1))
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_1 = nn.ReLU()
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_2 = nn.ReLU()
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_1 = nn.ReLU()
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_2 = nn.ReLU()
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_3 = nn.ReLU()
    self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
    
    # Output layer
    self.fc1 = nn.Linear(512 * 2 * 2, 512)
    self.act5 = nn.ReLU()
    self.out = nn.Linear(512, 10)
    
  def forward(self, x):
    x1 = x
    x2 = self.act1(self.conv1(x1))
    x3 = self.pool1(x2)
    
    x4 = self.act2(self.conv2(x3))
    x5 = self.pool2(x4)
    
    x6 = self.act3_1(self.conv3_1(x5))
    x7 = self.act3_2(self.conv3_2(x6))
    x8 = self.act3_3(self.conv3_3(x7))
    x9 = self.pool3(x8)
    
    x10 = self.act4_1(self.conv4_1(x9))
    x11 = self.act4_2(self.conv4_2(x10))
    x12 = self.act4_3(self.conv4_3(x11))
    x13 = self.pool4(x12)
    
    x14 = x13.view(-1, 512 * 2 * 2)
    
    x15 = self.act5(self.fc1(x14))
    
    out = self.out(x15)
    return out

In [0]:
class SimpleResNet(nn.Module):
  def __init__(self):
    super(SimpleResNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), padding=(1,1))
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    
    self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3),padding=(1,1))
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_1 = nn.ReLU()
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_2 = nn.ReLU()
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_1 = nn.ReLU()
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_2 = nn.ReLU()
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_3 = nn.ReLU()
    self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
    
    # Output layer
    self.fc1 = nn.Linear(512 * 2 * 2, 512)
    self.act5 = nn.ReLU()
    self.out = nn.Linear(512, 10)
  
  def forward(self, x):
    x1 = x
    x2 = self.act1(self.conv1(x1))
    x3 = self.pool1(x2)
    
    x4 = self.act2(self.conv2(x3))
    x5 = self.pool2(x4)
    
    x6 = self.act3_1(self.conv3_1(x5))
    x7 = self.act3_2(self.conv3_2(x6))
    x8 = self.act3_3(self.conv3_3(x7) + x6)
    x9 = self.pool3(x8)
    
    x10 = self.act4_1(self.conv4_1(x9))
    x11 = self.act4_1(self.conv4_2(x10))
    x12 = self.act4_1(self.conv4_3(x11) + x10)
    x13 = self.pool4(x12)
    
    x14 = x13.view(-1, 512 * 2 * 2)
    
    x15 = self.act5(self.fc1(x14))
    
    out = self.out(x15)
    return out           

In [66]:
mlp_model = SimpleMLP().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(mlp_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(mlp_model, input_size = (3, 32, 32))

1 loss : 2.1470139026641846
Accuracy : 0.6
2 loss : 1.0741104006767273
Accuracy : 0.5
3 loss : 1.2872629165649414
Accuracy : 0.5
4 loss : 0.8812382578849792
Accuracy : 0.5
5 loss : 0.8909328818321228
Accuracy : 0.5
6 loss : 0.6793242573738099
Accuracy : 0.55
7 loss : 0.4944648742675781
Accuracy : 0.5
8 loss : 0.6744342029094696
Accuracy : 0.8
9 loss : 0.4304735898971558
Accuracy : 0.75
10 loss : 0.2439924418926239
Accuracy : 0.75
11 loss : 0.2674423485994339
Accuracy : 0.6
12 loss : 0.1563424900174141
Accuracy : 0.8
13 loss : 0.10556281208992005
Accuracy : 0.8
14 loss : 0.10699136555194855
Accuracy : 0.8
15 loss : 0.0779783733189106
Accuracy : 0.8
16 loss : 0.22858626246452332
Accuracy : 0.45
17 loss : 0.6189836829900741
Accuracy : 0.8
18 loss : 0.17341877818107604
Accuracy : 0.8
19 loss : 0.08284797929227353
Accuracy : 0.75
20 loss : 0.057490409538149836
Accuracy : 0.8
21 loss : 0.3311044655740261
Accuracy : 0.8
22 loss : 0.09375998042523861
Accuracy : 0.75
23 loss : 0.046736373193562

In [67]:
mlp_model2 = SimpleMLP_Sigmoid().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(mlp_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(mlp_model2, input_size = (3, 32, 32))

1 loss : 7.543563697254285e-06
Accuracy : 0.8
2 loss : 7.543563879153226e-06
Accuracy : 0.8
3 loss : 7.524490501964465e-06
Accuracy : 0.8
4 loss : 7.534027099609375e-06
Accuracy : 0.8
5 loss : 7.514953585996409e-06
Accuracy : 0.8
6 loss : 7.514953722420614e-06
Accuracy : 0.8
7 loss : 7.486343474738533e-06
Accuracy : 0.8
8 loss : 7.476806513295742e-06
Accuracy : 0.8
9 loss : 7.495880163332913e-06
Accuracy : 0.8
10 loss : 7.457732999682776e-06
Accuracy : 0.8
11 loss : 7.4577332270564515e-06
Accuracy : 0.8
12 loss : 7.4481962656136606e-06
Accuracy : 0.8
13 loss : 7.438659713443485e-06
Accuracy : 0.8
14 loss : 7.438659622494015e-06
Accuracy : 0.8
15 loss : 7.43865966796875e-06
Accuracy : 0.8
16 loss : 7.43865966796875e-06
Accuracy : 0.8
17 loss : 7.419586199830519e-06
Accuracy : 0.8
18 loss : 7.4195862907799896e-06
Accuracy : 0.8
19 loss : 7.381439263554057e-06
Accuracy : 0.8
20 loss : 7.4005129135912284e-06
Accuracy : 0.8
21 loss : 7.38143924081669e-06
Accuracy : 0.8
22 loss : 7.362365840

In [68]:
cnn_model = SimpleCNN().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(cnn_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(cnn_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(cnn_model, input_size = (3,32,32))

1 loss : 2.0760456562042235
Accuracy : 0.5
2 loss : 1.5835627317428589
Accuracy : 0.5
3 loss : 1.2070239067077637
Accuracy : 0.5
4 loss : 1.381605863571167
Accuracy : 0.5
5 loss : 0.8556420564651489
Accuracy : 0.5
6 loss : 0.7671935796737671
Accuracy : 0.5
7 loss : 0.9290951132774353
Accuracy : 0.5
8 loss : 0.7881680250167846
Accuracy : 0.5
9 loss : 0.7248735070228577
Accuracy : 0.5
10 loss : 0.7248119831085205
Accuracy : 0.5
11 loss : 0.7110259175300598
Accuracy : 0.5
12 loss : 0.7099810719490052
Accuracy : 0.5
13 loss : 0.777424156665802
Accuracy : 0.5
14 loss : 0.7380807518959045
Accuracy : 0.5
15 loss : 0.7586222171783448
Accuracy : 0.5
16 loss : 0.8086974024772644
Accuracy : 0.65
17 loss : 0.6614022612571716
Accuracy : 0.55
18 loss : 0.716345465183258
Accuracy : 0.5
19 loss : 0.7388098835945129
Accuracy : 0.75
20 loss : 0.6093854308128357
Accuracy : 0.75
21 loss : 0.5151138842105866
Accuracy : 0.55
22 loss : 0.7804114758968353
Accuracy : 0.5
23 loss : 0.8038095831871033
Accuracy :

In [69]:
vgg_model = SimpleVGG().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(vgg_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(vgg_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(vgg_model, input_size = (3,32,32))

1 loss : 2.208039951324463
Accuracy : 0.5
2 loss : 1.9862875938415527
Accuracy : 0.5
3 loss : 1.7810494661331178
Accuracy : 0.5
4 loss : 1.5791258335113525
Accuracy : 0.5
5 loss : 1.3629652738571167
Accuracy : 0.5
6 loss : 1.1400715112686157
Accuracy : 0.5
7 loss : 0.9319798946380615
Accuracy : 0.5
8 loss : 0.7931546092033386
Accuracy : 0.5
9 loss : 0.7415616154670716
Accuracy : 0.5
10 loss : 0.7209185719490051
Accuracy : 0.5
11 loss : 0.7321910500526428
Accuracy : 0.5
12 loss : 0.7724580287933349
Accuracy : 0.5
13 loss : 0.7427847862243653
Accuracy : 0.5
14 loss : 0.7403046488761902
Accuracy : 0.5
15 loss : 0.7252024173736572
Accuracy : 0.5
16 loss : 0.7348765134811401
Accuracy : 0.5
17 loss : 0.7406313419342041
Accuracy : 0.5
18 loss : 0.7083437085151673
Accuracy : 0.5
19 loss : 0.7048942446708679
Accuracy : 0.5
20 loss : 0.7042027473449707
Accuracy : 0.5
21 loss : 0.8282113790512085
Accuracy : 0.5
22 loss : 0.7208207249641418
Accuracy : 0.5
23 loss : 0.7325517892837524
Accuracy : 0.

In [70]:
resnet_model = SimpleResNet().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(resnet_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(resnet_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(resnet_model, input_size = (3,32,32))

1 loss : 2.1406479358673094
Accuracy : 0.5
2 loss : 1.825917088985443
Accuracy : 0.5
3 loss : 1.8171675682067872
Accuracy : 0.5
4 loss : 1.424516201019287
Accuracy : 0.5
5 loss : 0.8961857795715332
Accuracy : 0.5
6 loss : 0.7483895897865296
Accuracy : 0.5
7 loss : 0.7531236171722412
Accuracy : 0.5
8 loss : 0.7378690123558045
Accuracy : 0.5
9 loss : 0.7510707378387451
Accuracy : 0.5
10 loss : 0.7074931263923645
Accuracy : 0.5
11 loss : 0.7370277762413024
Accuracy : 0.65
12 loss : 0.6928037405014038
Accuracy : 0.5
13 loss : 0.7330654978752136
Accuracy : 0.5
14 loss : 0.7933225512504578
Accuracy : 0.8
15 loss : 0.6780535459518433
Accuracy : 0.5
16 loss : 0.6490682244300843
Accuracy : 0.7
17 loss : 0.7173475742340087
Accuracy : 0.5
18 loss : 0.7430318176746369
Accuracy : 0.65
19 loss : 0.6842571377754212
Accuracy : 0.55
20 loss : 0.6014569878578186
Accuracy : 0.65
21 loss : 0.6327221870422364
Accuracy : 0.65
22 loss : 0.5032673835754394
Accuracy : 0.8
23 loss : 0.6241871237754821
Accuracy 